In [ ]:
!pip install fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pyngrok

In [ ]:
!pip install nltk

In [ ]:
# Download NLTK resources
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [ ]:
# from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
# import torch
# from PIL import Image
# import nltk
# from nltk import pos_tag
# from nltk.tokenize import word_tokenize

# # Download NLTK resources
# # nltk.download('punkt')
# # nltk.download('averaged_perceptron_tagger')

# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# max_length = 16
# num_beams = 4
# gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# def extract_nouns(text):
#     tokens = word_tokenize(text)
#     tagged = pos_tag(tokens)
#     nouns = [word for word, pos in tagged if pos.startswith('NN')]
#     return nouns

# def hash_pred(image):
#     images = [image]

#     pixel_values = feature_extractor(
#         images=images, return_tensors="pt"
#     ).pixel_values
#     pixel_values = pixel_values.to(device)
#     output_ids = model.generate(pixel_values, **gen_kwargs)

#     preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
#     preds = [pred.strip() for pred in preds]
#     print("Generated caption:", preds)

#     hashtags = []
#     for caption in preds:
#         nouns = extract_nouns(caption)
#         for noun in nouns:
#             hashtags.append('#' + noun)

#     print("Generated hashtags:", hashtags)
#     res = {}
#     res['preds'] = preds
#     res['hashtags'] = hashtags
#     return res

In [ ]:
import uvicorn
import nest_asyncio


In [ ]:
from fastapi import FastAPI, File, UploadFile
from PIL import Image
import io
from pydantic import BaseModel


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


def cap_pred(input_text):
    max_length = 128
    input_ids = tokenizer(input_text, return_tensors="pt")

    input_ids = input_ids['input_ids'].to(device)

    output = model.generate(input_ids, max_length=max_length, num_beams=5,
                        do_sample=False, no_repeat_ngram_size=2)
    captions=tokenizer.decode(output[0], skip_special_tokens=True)

    return captions

import requests
# from model import hash_pred
# from model2 import cap_pred
import uvicorn

# Initialize FastAPI app
app = FastAPI()

class RequestPrediction(BaseModel):
    file: str

class RequestCaptions(BaseModel):
    text: str

# Example function to process image file and make predictions
def process_image(fileUrl):

    response = requests.get(fileUrl)

    image = Image.open(io.BytesIO(response.content))

    predictions = hash_pred(image)
    return predictions

def process_text(desc):
    captions = cap_pred(desc)
    return captions

# Define endpoint to receive image file
@app.post("/predict/")
async def predict(req: RequestPrediction):
    predictions = process_image(req.file)
    print(predictions)
    return {"predictions": predictions}

@app.post("/captions/")
async def captions(req: RequestCaptions):
    captions = process_text(req.text)
    return {"captions" : captions}



# uvicorn.run(app=app, port=8000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Setup a tunnel to the FastAPI server
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app=app, port=8000)

INFO:     Started server process [316]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://b8b1-34-125-51-248.ngrok-free.app


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     150.129.64.98:0 - "POST /captions/ HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

INFO:     150.129.64.98:0 - "POST /captions/ HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     150.129.64.98:0 - "POST /captions/ HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     150.129.64.98:0 - "POST /captions/ HTTP/1.1" 200 OK
